### **6.8 - What if a task needs the output of its previous execution?**

Hay un argumento que puedes definir para cualquier operador en airflow, que se llama "depends_on_past". Y este argumento es extremadamente útil, pero a veces bastante confuso. Así que déjame decirte exactamente lo que es. En primer lugar, tienes que recordar que el objetivo de "depends_on_past" es evitar que se ejecute una tarea en un DagRun si la misma tarea no tuvo éxito en el DagRun anterior. Déjame mostrarte un ejemplo concreto. Entonces, digamos que tienes esos dos DagRuns, con un DAG muy simple, solo tres tareas "A-B-C", y "depends_on_past" está definido para esta tarea:

<center><img src="https://i.postimg.cc/4d5xTQ5Q/a1432.png"></center>

Digamos que la tarea A en DagRun 1 no tuvo éxito, por lo que falló. En ese caso, tan pronto como se active DagRun 2, la tarea A no será disparada por el scheduler.

<center><img src="https://i.postimg.cc/BbnQZP79/a1433.png"></center>

¿Por qué? Porque la tarea A depende de la ejecución previa de la misma tarea en el DagRun anterior. 

Otro ejemplo, si usted dijo "depends_on_past" en la tarea B y la tarea B no tuvo éxito en DagRun 1, de nuevo en DagRun 2, la tarea B no se disparará. Pero, la tarea A en DagRun 2 se disparará, ¿verdad? Porque la tarea A no tiene "depends_on_past".

<center><img src="https://i.postimg.cc/J7pz6Kt1/a1434.png"></center>

Ahora, que pasa si tienes "depends_on_past" en todas las tareas, para "A-B y C". Significa que el DagRun 2 se activará sólo si el DagRun anterior (DagRun 1) ha tenido éxito. Bueno, absolutamente no, y esta es la primera cosa confusa que puedes escuchar sobre "depends_on_past". Si dices "depends_on_past" en todas las tareas de tu DAG, bien, una cosa con la que terminarás es que el segundo DagRun se disparará y entonces Airflow comprobará si la primera tarea, la tarea A ha tenido éxito en el DagRun anterior. 

<center><img src="https://i.postimg.cc/nrSVXVNs/a1435.png"></center>

si es así, entonces la tarea A se ejecutará en el segundo DagRun 

<center><img src="https://i.postimg.cc/J4g4mnbv/a1436.png"></center>

y, a continuación, para la segunda tarea, la tarea B, Airflow comprobará si en el DagRun anterior la tarea B ha tenido éxito. 

<center><img src="https://i.postimg.cc/PqnXpRVM/a1437.png"></center>

Si es así, la tarea B se disparará y así sucesivamente.

<center><img src="https://i.postimg.cc/0yw5NsnQ/a1438.png"></center>

Así que ten en cuenta que "depends_on_past" no impide que se ejecute el siguiente DagRun si el DagRun anterior no tuvo éxito. Este no es en absoluto el objetivo de "depends_on_past". Y una cosa a tener en cuenta, es que, si tu tarea no se dispara porque la instancia de la tarea anterior no tuvo éxito, tu tarea no terminará con ningún estado. Simplemente tu tarea no se disparará en absoluto. La tarea no tendrá ningún estado. Por eso es muy importante establecer siempre un timeout para tus tareas, porque de lo contrario te quedarás atascado. Dicho esto, vamos a ver cómo utilizar "depends_on_past" en la vida real. En tu DAG quieres asegurarte de que las tareas "extract_partners_snowflake", "extract_partners_netflix" y "extract_partners_astronomer" han tenido éxito en el DagRun anterior para ejecutarlas en el DagRun actual. Para hacer eso, necesitas especificar "depends_on_past", en cada tarea. Hagamos esto ahora mismo, así que vuelve a tu editor de código y aquí como argumento de tu tarea, especifica "depends_on_past" igual a "True". 

<center><img src="https://i.postimg.cc/fb7wxf6Q/a1439.png"></center>

Guarda el archivo y ya está, volvamos a la interfaz de usuario de Airflow, activemos el toggle del DAG para ver qué pasa, actualiza la página y todo se activa como se esperaba.

<center><img src="https://i.postimg.cc/m2h20MF7/a1440.png"></center>

Ahora el DAG se ha completado. Eso está bien. Ahora, ¿qué pasa si esas tareas han fallado en lugar de tener éxito? Para simular esto, puedes escribir "raise ValueError('failed') sólo para simular un error:

<center><img src="https://i.postimg.cc/MTWztjSM/a1441.png"></center>

y asegúrate de que en los default arguments tienes un retries igual a "0" de lo contrario Airflow reintentará tus tareas:

<center><img src="https://i.postimg.cc/xCtnZTfs/a1442.png"></center>

Así que vamos a guardar el archivo y volver a la interfaz de usuario de Airflow, reintentar el DAG de nuevo, así que haz clic en la tarea "start", luego en "Clear".

<center><img src="https://i.postimg.cc/900CH5zc/a1443.png"></center>

Refresca la página y espera un poco, como puedes ver las tareas han fallado y ahora como tienes "depends_on_past" configurado para las tareas "extract_partner", si activas un segundo DagRun, esas tareas (las tareas "extract_partner") no deberían activarse. 

<center><img src="https://i.postimg.cc/QNYjktnC/a1444.png"></center>

Veamos esto, dispara el DAG manualmente

<center><img src="https://i.postimg.cc/5yr1x7FL/a1445.png"></center>

Volvamos al Graph View, esperemos un poco, y si refrescamos la página, pues no pasa nada. 

<center><img src="https://i.postimg.cc/KvdFwWby/a1446.png"></center>
<center><img src="https://i.postimg.cc/c4K0DSVm/a1447.png"></center>

Ese es el comportamiento de "depends_on_past". Como puedes ver, no termina con un fallo o algo así. Nada. Simplemente tus tareas no tienen ningún estado. Y de nuevo, es por eso que siempre debes especificar un "timeout" de lo contrario, tu DagRun seguirá corriendo para siempre. Finalmente, solo unas notas adicionales sobre "depends_on_past". La primera es que, si disparas tu DagRun manualmente, entonces "depends_on_past" será evaluado. Funciona también para DagRuns disparados manualmente, obviamente para el primer DagRun de tu DAG, "depends_on_past" no se evalúa, de lo contrario, no serás capaz de ejecutar tu DAG en absoluto si rellenas (backfill) tu DAG en ese caso, la "start_date" será sobrescrita por la "star_date" dada en el backfill job, de lo contrario de nuevo, no serás capaz de rellenar (backfill) tu DAG. Así es como funciona. Por último, pero no menos importante, "depends_on_past" no sólo dispara tu tarea si la misma tarea en el DagRun anterior ha tenido éxito, sino también, si se ha omitido. Así que recuerda esto, "depends_on_past" ejecuta tu tarea si la misma tarea ha tenido éxito o ha sido omitida en el DagRun anterior. Y ahora sabes todo lo que necesitas sobre "depends_on_past" y eres capaz de crear dependencias para la misma tarea entre diferentes DagRuns.